**Importing libraries**

In [1]:
import pandas as pd
import datetime
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Reading and understanding the data
#link = "https://data.seattle.gov/api/views/tazs-3rd5/rows.csv?date=20231107&accessType=DOWNLOAD"

#Reading
seattle_crime = pd.read_csv('/content/drive/MyDrive/SPD_Crime_Data__2008-Present_20231027.csv')

In [4]:
seattle_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080712 entries, 0 to 1080711
Data columns (total 17 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   Report Number           1080712 non-null  object 
 1   Offense ID              1080712 non-null  int64  
 2   Offense Start DateTime  1079241 non-null  object 
 3   Offense End DateTime    614173 non-null   object 
 4   Report DateTime         1080712 non-null  object 
 5   Group A B               1080712 non-null  object 
 6   Crime Against Category  1080712 non-null  object 
 7   Offense Parent Group    1080712 non-null  object 
 8   Offense                 1080712 non-null  object 
 9   Offense Code            1080712 non-null  object 
 10  Precinct                1080694 non-null  object 
 11  Sector                  1080696 non-null  object 
 12  Beat                    1080696 non-null  object 
 13  MCPP                    1080698 non-null  object 
 14  10

## Data Preprocessing

In [5]:
seattle_crime.isnull().sum()

Report Number                  0
Offense ID                     0
Offense Start DateTime      1471
Offense End DateTime      466539
Report DateTime                0
Group A B                      0
Crime Against Category         0
Offense Parent Group           0
Offense                        0
Offense Code                   0
Precinct                      18
Sector                        16
Beat                          16
MCPP                          14
100 Block Address          47239
Longitude                      0
Latitude                       0
dtype: int64

In [6]:
#Treating missing values
#Removing Offense End DateTime and 100 Block Address as we have other columns to support these info.
#Along removing other missing values
sc=seattle_crime.drop(['Offense End DateTime'], axis=1)
sc = sc.drop(['100 Block Address'],axis=1)
sc = sc.dropna()
sc.isnull().sum()

Report Number             0
Offense ID                0
Offense Start DateTime    0
Report DateTime           0
Group A B                 0
Crime Against Category    0
Offense Parent Group      0
Offense                   0
Offense Code              0
Precinct                  0
Sector                    0
Beat                      0
MCPP                      0
Longitude                 0
Latitude                  0
dtype: int64

In [7]:
#Converting date and time colume
#copying the values to a new column
sc['Offense DateTime'] = sc['Offense Start DateTime']
#converting to datetime datatype
sc['Offense Start DateTime'] = pd.to_datetime(sc['Offense Start DateTime'], format='%m/%d/%Y %I:%M:%S %p')
#fetching year
sc['year']=sc['Offense Start DateTime'].dt.year

#considering required data from 2008 to 2022
sc = sc[sc['year'] >= 2021]
sc = sc[sc['year'] <= 2022]

#mcpp
sc = sc[sc['MCPP'] != '<Null>']
sc = sc[sc['MCPP'] != 'UNKNOWN']


In [8]:
sc1 = sc[['MCPP','Offense Parent Group','Offense Start DateTime']]

In [9]:
# Ensure that 'CrimeType' is a categorical variable (if not, convert it to one)
label_encoder = LabelEncoder().fit(sc1['Offense Parent Group'])
sc1['Offense Parent Group'] = label_encoder.transform(sc1['Offense Parent Group'])

# One-hot encode the 'MCPP' column
sc1 = pd.get_dummies(sc1, columns=['MCPP'], prefix='MCPP', drop_first=True)

In [10]:
X = sc1.drop(['Offense Parent Group','Offense Start DateTime'], axis=1)
y = sc1['Offense Parent Group']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Random forest**

In [11]:
# RandomForestClassifier Training
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
# RandomForestClassifier Prediction
rf_y_pred = rf_model.predict(X_test)
# Calculate accuracy on the test set for RandomForestClassifier
rf_accuracy = accuracy_score(y_test, rf_y_pred)
print(f"Random Forest Accuracy on the test set: {rf_accuracy}")
# Classification report for RandomForestClassifier
rf_report = classification_report(y_test, rf_y_pred)
print("Random Forest Classification Report:")
print(rf_report)
# Predict the probabilities for each class with RandomForestClassifier
rf_y_probs = rf_model.predict_proba(X_test)
# Get the top predicted crime type and its probability for each MCPP with RandomForestClassifier
rf_predictions = []

for i, row in enumerate(X_test.itertuples()):
    # Identify the MCPP columns
    rf_mcpp_cols = [col for col, value in zip(X_test.columns, row[1:]) if value == 1]

    if rf_mcpp_cols:
        rf_mcpp = rf_mcpp_cols[0].replace('MCPP_', '')  # Remove the 'MCPP_' prefix
        rf_top_crime_index = rf_y_probs[i].argmax()
        rf_top_crime_type = label_encoder.inverse_transform([rf_top_crime_index])[0]
        rf_top_crime_probability = rf_y_probs[i][rf_top_crime_index]

        rf_predictions.append((row[0], rf_mcpp, rf_top_crime_type, rf_top_crime_probability))

# Display predictions for RandomForestClassifier
rf_predictions_df = pd.DataFrame(rf_predictions, columns=['Timestamp', 'MCPP', 'Top Crime Type', 'Probability'])

# Reorder the columns
rf_predictions_df = rf_predictions_df[['MCPP', 'Top Crime Type', 'Probability']]

# Remove duplicate rows based on 'MCPP' column for RandomForestClassifier
rf_predictions_df = rf_predictions_df.drop_duplicates(subset=['MCPP'])

print(rf_predictions_df)

Random Forest Accuracy on the test set: 0.38180490857262045
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.00      0.00      0.00        84
           2       0.30      0.03      0.05      4422
           3       0.00      0.00      0.00         9
           5       0.00      0.00      0.00      3702
           6       0.00      0.00      0.00       118
           7       0.00      0.00      0.00         7
           8       0.00      0.00      0.00      2819
           9       0.00      0.00      0.00       451
          10       0.00      0.00      0.00       211
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00        62
          14       0.00      0.00      0.00        33
          15       0.00      0.00      0.00      1073
          16       0.00      0.00     

Analysis of RF model:

### Random Forest Model Performance:
- **Overall Accuracy**: The accuracy on the test set is 0.3818, indicating that approximately 38.18% of the predictions made by the Random Forest model are correct.

- **Classification Report**:
  - The report shows a significant variance in performance across different crime types (labeled from 0 to 31).
  - For many crime types, both precision and recall are 0.00, suggesting the model fails to correctly identify these crime types either due to lack of representative data in these classes or model's inability to learn distinguishing features for these classes.
  - For crime type `20`, the model shows a substantially better performance with a precision of 0.38 and a recall of 0.99. This indicates that the model is highly successful in identifying this particular crime type, but it also suggests a possible class imbalance where the model might be biased towards predicting this class more frequently.
  - The weighted average precision, recall, and F1-score across all classes are relatively low, highlighting the need for model improvement or reconsideration of the feature set or data.

### Predictions for Different MCPP Areas:
- The predictions for various MCPP areas predominantly show "KIDNAPPING/ABDUCTION" as the top predicted crime type, with varying probabilities.






**Logistic Regression**

In [12]:


# Logistic Regression Training
lr_model = LogisticRegression(random_state=42)
lr_model.fit(X_train, y_train)

# Logistic Regression Prediction
lr_y_pred = lr_model.predict(X_test)

# Calculate accuracy on the test set for Logistic Regression
lr_accuracy = accuracy_score(y_test, lr_y_pred)
print(f"Logistic Regression Accuracy on the test set: {lr_accuracy}")

# Classification report for Logistic Regression
lr_report = classification_report(y_test, lr_y_pred)
print("Logistic Regression Classification Report:")
print(lr_report)

# Predict the probabilities for each class with Logistic Regression
lr_y_probs = lr_model.predict_proba(X_test)

# Get the top predicted crime type and its probability for each MCPP with Logistic Regression
lr_predictions = []

for i, row in enumerate(X_test.itertuples()):
    lr_mcpp_cols = [col for col, value in zip(X_test.columns, row[1:]) if value == 1]

    if lr_mcpp_cols:
        lr_mcpp = lr_mcpp_cols[0].replace('MCPP_', '')
        lr_top_crime_index = lr_y_probs[i].argmax()
        lr_top_crime_type = label_encoder.inverse_transform([lr_top_crime_index])[0]
        lr_top_crime_probability = lr_y_probs[i][lr_top_crime_index]

        lr_predictions.append((row[0], lr_mcpp, lr_top_crime_type, lr_top_crime_probability))

# Display predictions for Logistic Regression
lr_predictions_df = pd.DataFrame(lr_predictions, columns=['Timestamp', 'MCPP', 'Top Crime Type', 'Probability'])
lr_predictions_df = lr_predictions_df[['MCPP', 'Top Crime Type', 'Probability']]
lr_predictions_df = lr_predictions_df.drop_duplicates(subset=['MCPP'])
print(lr_predictions_df)


Logistic Regression Accuracy on the test set: 0.381839671834805
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.00      0.00      0.00        84
           2       0.30      0.04      0.07      4422
           3       0.00      0.00      0.00         9
           5       0.00      0.00      0.00      3702
           6       0.00      0.00      0.00       118
           7       0.00      0.00      0.00         7
           8       0.00      0.00      0.00      2819
           9       0.00      0.00      0.00       451
          10       0.00      0.00      0.00       211
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00        62
          14       0.00      0.00      0.00        33
          15       0.00      0.00      0.00      1073
          16       0.00     

**Logistic Regression Model Performance:**

**Overall Accuracy:** The model's accuracy on the test set is 0.3818, indicating that approximately 38.18% of predictions are correct.

**Classification Report:**
Similar to the Random Forest model, there's a significant variance in performance across different crime types.
The precision and recall for most crime types are 0.00, indicating the model is unable to correctly predict these crime types. This could be due to insufficient or non-representative data for these classes, or the model's limitations in capturing the complexity of the data.

Crime type 20 shows a markedly better performance with high recall (0.99) but moderate precision (0.38), suggesting a possible class imbalance where the model is predominantly predicting this class.

The weighted average for precision, recall, and F1-score are modest, highlighting the need for improvements in the model.

**Predictions for Different MCPP Areas:**
The predictions across various MCPP areas primarily indicate "KIDNAPPING/ABDUCTION" as the top predicted crime type. This is consistent with the high recall observed for crime type 20 in the classification report.

The consistent prediction of a single crime type across multiple areas might point to overfitting or class imbalance, where the model has learned to preferentially predict a dominant class present in the training data.
The probabilities associated with these predictions vary, but the lack of diversity in the predicted crime types is a point of concern.

**Gradient Boosting**

In [13]:
# Gradient Boosting Training
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)

# Gradient Boosting Prediction
gb_y_pred = gb_model.predict(X_test)

# Calculate accuracy on the test set for Gradient Boosting
gb_accuracy = accuracy_score(y_test, gb_y_pred)
print(f"Gradient Boosting Accuracy on the test set: {gb_accuracy}")

# Classification report for Gradient Boosting
gb_report = classification_report(y_test, gb_y_pred)
print("Gradient Boosting Classification Report:")
print(gb_report)

# Predict the probabilities for each class with Gradient Boosting
gb_y_probs = gb_model.predict_proba(X_test)

# Get the top predicted crime type and its probability for each MCPP with Gradient Boosting
gb_predictions = []

for i, row in enumerate(X_test.itertuples()):
    gb_mcpp_cols = [col for col, value in zip(X_test.columns, row[1:]) if value == 1]

    if gb_mcpp_cols:
        gb_mcpp = gb_mcpp_cols[0].replace('MCPP_', '')
        gb_top_crime_index = gb_y_probs[i].argmax()
        gb_top_crime_type = label_encoder.inverse_transform([gb_top_crime_index])[0]
        gb_top_crime_probability = gb_y_probs[i][gb_top_crime_index]

        gb_predictions.append((row[0], gb_mcpp, gb_top_crime_type, gb_top_crime_probability))

# Display predictions for Gradient Boosting
gb_predictions_df = pd.DataFrame(gb_predictions, columns=['Timestamp', 'MCPP', 'Top Crime Type', 'Probability'])
gb_predictions_df = gb_predictions_df[['MCPP', 'Top Crime Type', 'Probability']]
gb_predictions_df = gb_predictions_df.drop_duplicates(subset=['MCPP'])
print(gb_predictions_df)


Gradient Boosting Accuracy on the test set: 0.38110964332893
Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.00      0.00      0.00        84
           2       0.00      0.00      0.00      4422
           3       0.00      0.00      0.00         9
           5       0.00      0.00      0.00      3702
           6       0.00      0.00      0.00       118
           7       0.00      0.00      0.00         7
           8       0.00      0.00      0.00      2819
           9       0.00      0.00      0.00       451
          10       0.00      0.00      0.00       211
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00        62
          14       0.00      0.00      0.00        33
          15       0.00      0.00      0.00      1073
          16       0.00      0.00

**Gradient Boosting Model Performance:**

**Overall Accuracy:** The accuracy on the test set is 0.3811, similar to the other models, indicating around 38.11% of the predictions are correct.

**Classification Report:**

The precision and recall for most crime types are 0.00, showing the model's inability to predict these crime types effectively. This could be due to various reasons, such as lack of representative data for these classes or the model's limitations.

The model performs notably better for crime type 20 with high recall (1.00) and moderate precision (0.38). This suggests a significant class imbalance where the model might be overfitting to this particular crime type.
The weighted average metrics (precision, recall, and F1-score) are low, indicating the model is not performing well overall in differentiating between the various crime types.

**Predictions for Different MCPP Areas:**
The model's predictions are predominantly "KIDNAPPING/ABDUCTION" for most MCPP areas, with varying probabilities. This is consistent with the high recall for crime type 20.

The consistent prediction of a single crime type across many areas might suggest class imbalance or overfitting to dominant classes in the training data.
For one MCPP area (COMMERCIAL HARBOR ISLAND), the model predicts a different crime type ("ROBBERY") with high probability.

**Decision Tree**

In [14]:


# Decision Tree Training
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Decision Tree Prediction
dt_y_pred = dt_model.predict(X_test)

# Calculate accuracy on the test set for Decision Tree
dt_accuracy = accuracy_score(y_test, dt_y_pred)
print(f"Decision Tree Accuracy on the test set: {dt_accuracy}")

# Classification report for Decision Tree
dt_report = classification_report(y_test, dt_y_pred)
print("Decision Tree Classification Report:")
print(dt_report)

# Predict the probabilities for each class with Decision Tree
dt_y_probs = dt_model.predict_proba(X_test)

# Get the top predicted crime type and its probability for each MCPP with Decision Tree
dt_predictions = []

for i, row in enumerate(X_test.itertuples()):
    dt_mcpp_cols = [col for col, value in zip(X_test.columns, row[1:]) if value == 1]

    if dt_mcpp_cols:
        dt_mcpp = dt_mcpp_cols[0].replace('MCPP_', '')
        dt_top_crime_index = dt_y_probs[i].argmax()
        dt_top_crime_type = label_encoder.inverse_transform([dt_top_crime_index])[0]
        dt_top_crime_probability = dt_y_probs[i][dt_top_crime_index]

        dt_predictions.append((row[0], dt_mcpp, dt_top_crime_type, dt_top_crime_probability))

# Display predictions for Decision Tree
dt_predictions_df = pd.DataFrame(dt_predictions, columns=['Timestamp', 'MCPP', 'Top Crime Type', 'Probability'])
dt_predictions_df = dt_predictions_df[['MCPP', 'Top Crime Type', 'Probability']]
dt_predictions_df = dt_predictions_df.drop_duplicates(subset=['MCPP'])
print(dt_predictions_df)


Decision Tree Accuracy on the test set: 0.381839671834805
Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.00      0.00      0.00        84
           2       0.30      0.04      0.07      4422
           3       0.00      0.00      0.00         9
           5       0.00      0.00      0.00      3702
           6       0.00      0.00      0.00       118
           7       0.00      0.00      0.00         7
           8       0.00      0.00      0.00      2819
           9       0.00      0.00      0.00       451
          10       0.00      0.00      0.00       211
          11       0.00      0.00      0.00         2
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00        62
          14       0.00      0.00      0.00        33
          15       0.00      0.00      0.00      1073
          16       0.00      0.00      0

**Decision Tree Model Performance:**

**Overall Accuracy:** The accuracy on the test set is 0.3818, which means around 38.18% of the predictions made by the Decision Tree model are correct.

**Classification Report:**
Similar to the previous models, there's a significant variance in performance across different crime types.
The precision and recall for most crime types are 0.00, showing the model's inability to effectively predict these crime types. This could be due to various reasons such as lack of representative data for these classes, or the model's limitations.
The model shows better performance for crime type 20 with high recall (0.99) and moderate precision (0.38), suggesting a potential class imbalance issue.
The weighted average for precision, recall, and F1-score are low, indicating the model is not performing well overall in differentiating between the various crime types.

In [20]:
from tabulate import tabulate

# List of model predictions
model_predictions = {
    'Random Forest': rf_y_pred,
    'Logistic Regression': lr_y_pred,
    'Gradient Boosting': gb_y_pred,
    'Decision Tree': dt_y_pred
}

# Initialize a DataFrame to store the metrics
metrics_df = pd.DataFrame(columns=['Model', 'Precision', 'Recall', 'F1-Score'])

# Calculate and store precision, recall, and F1-score for each model
for model_name, predictions in model_predictions.items():
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, predictions, average='weighted')
    metrics_df = metrics_df.append({'Model': model_name, 'Precision': precision, 'Recall': recall, 'F1-Score': f1_score}, ignore_index=True)

# Display the metrics DataFrame using tabulate
print("Performance Metrics Before Cross-Validation:")
print(tabulate(metrics_df, headers='keys', tablefmt='grid'))




Performance Metrics Before Cross-Validation:
+----+---------------------+-------------+----------+------------+
|    | Model               |   Precision |   Recall |   F1-Score |
+====+=====================+=============+==========+============+
|  0 | Random Forest       |    0.192594 | 0.381805 |   0.217893 |
+----+---------------------+-------------+----------+------------+
|  1 | Logistic Regression |    0.192523 | 0.38184  |   0.221228 |
+----+---------------------+-------------+----------+------------+
|  2 | Gradient Boosting   |    0.145435 | 0.38111  |   0.210529 |
+----+---------------------+-------------+----------+------------+
|  3 | Decision Tree       |    0.192523 | 0.38184  |   0.221228 |
+----+---------------------+-------------+----------+------------+



1. **Random Forest**:
   - **Precision: 0.192594** - This suggests that when the Random Forest model predicts a certain crime type, it is correct approximately 19.26% of the time. This is relatively low, indicating a high rate of false positives.
   - **Recall: 0.381805** - About 38.18% of actual crimes of a specific type are correctly identified by the model. This suggests moderate sensitivity.
   - **F1-Score: 0.217893** - The F1-Score balances precision and recall, and a score of approximately 0.218 indicates moderate overall accuracy, but with room for improvement.

2. **Logistic Regression**:
   - **Precision: 0.192523** - Similar to Random Forest, Logistic Regression is correct about 19.25% of the time when predicting a crime type.
   - **Recall: 0.381840** - This model also correctly identifies about 38.18% of the actual crimes, indicating a similar recall to Random Forest.
   - **F1-Score: 0.221228** - The slightly higher F1-Score compared to Random Forest suggests a marginally better balance between precision and recall.

3. **Gradient Boosting**:
   - **Precision: 0.145435** - This model has a lower precision, being correct only about 14.54% of the time, indicating more false positives compared to the other models.
   - **Recall: 0.381110** - Its ability to identify actual crimes is comparable to the other models (approximately 38.11%).
   - **F1-Score: 0.210529** - The overall balance between precision and recall is slightly lower than the other models.

4. **Decision Tree**:
   - **Precision: 0.192523** - Identical to Logistic Regression, indicating similar rates of false positives.
   - **Recall: 0.381840** - The same recall as Logistic Regression, suggesting equal effectiveness in identifying actual crimes.
   - **F1-Score: 0.221228** - This score is identical to Logistic Regression, indicating a similar overall performance.

### Crime Type Prediction Task Analysis:

- **Recall Across Models**: All models have similar recall scores, suggesting they are equally capable of identifying actual crimes. However, the recall is less than 50%, indicating that more than half of the actual crimes are not being detected by any of the models.

- **Precision and False Positives**: The precision is low for all models, especially for Gradient Boosting. This implies that there are a significant number of false positives – instances where the models incorrectly predict a crime type.

- **Overall Accuracy (F1-Score)**: The F1-Scores are moderate but suggest there is significant room for improvement. The similar F1-Scores of Logistic Regression and Decision Tree might indicate that these models are striking a similar balance between missing actual crimes and incorrectly predicting crimes that did not happen.

For our crime type prediction task, these metrics suggest that while the models are somewhat effective in identifying crimes (as shown by recall), they are not very precise, leading to many false alarms (as shown by precision).

**Applying cross validation**

In [21]:
 from sklearn.model_selection import cross_validate
from tabulate import tabulate

# Define the number of folds for cross-validation
num_folds = 5

# Define the scoring metrics
scoring_metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# Initialize a dictionary to store the results
results = {'Model': [], 'Mean Accuracy': [], 'Mean Precision': [], 'Mean Recall': [], 'Mean F1 Score': []}

# Perform cross-validation and store the results
for model_name, model in [('Random Forest', rf_model), ('Logistic Regression', lr_model), ('Gradient Boosting', gb_model), ('Decision Tree', dt_model)]:
    cv_results = cross_validate(model, X_train, y_train, cv=num_folds, scoring=scoring_metrics)
    results['Model'].append(model_name)
    results['Mean Accuracy'].append(cv_results['test_accuracy'].mean())
    results['Mean Precision'].append(cv_results['test_precision_macro'].mean())
    results['Mean Recall'].append(cv_results['test_recall_macro'].mean())
    results['Mean F1 Score'].append(cv_results['test_f1_macro'].mean())

# Display the results in tabular form
print(tabulate(results, headers='keys', tablefmt='grid'))


+---------------------+-----------------+------------------+---------------+-----------------+
| Model               |   Mean Accuracy |   Mean Precision |   Mean Recall |   Mean F1 Score |
+=====================+=================+==================+===============+=================+
| Random Forest       |        0.381423 |        0.0225129 |     0.0338142 |       0.0202627 |
+---------------------+-----------------+------------------+---------------+-----------------+
| Logistic Regression |        0.381431 |        0.0227025 |     0.0337685 |       0.0201375 |
+---------------------+-----------------+------------------+---------------+-----------------+
| Gradient Boosting   |        0.380901 |        0.0166017 |     0.0333101 |       0.0188924 |
+---------------------+-----------------+------------------+---------------+-----------------+
| Decision Tree       |        0.381423 |        0.0225129 |     0.0338142 |       0.0202627 |
+---------------------+-----------------+---------

After cross-validation, all models exhibit low precision, recall, and F1 scores. The Random Forest and Decision Tree models have identical metrics, and the Logistic Regression model shows similar performance. Gradient Boosting has slightly lower scores across all metrics. This uniformity in results, particularly the low precision and F1 scores, suggests a need for further model tuning.


**Understanding the Challenge of Predicting Crime with Data:**
Crime prediction models face complex challenges due to the multifaceted nature of crime. Various factors such as socio-economic conditions, demographic shifts, and geographic elements interact in unpredictable ways, making accurate predictions difficult.

**Rationale Behind Model Selection:**
Our choice of models - RandomForest, Logistic Regression, Gradient Boosting, and Decision Tree - was based on their diverse capabilities in pattern recognition and data handling. Each model brings a unique approach to the table, allowing us to explore different angles of the data.

**The Role of Cross-Validation:**
We employed cross-validation to ensure a thorough and unbiased evaluation of each model's performance. This method provides insights into how these models fare across various data segments, enhancing the reliability of our results.

**Interpreting Our Metrics:**
The observed low precision, recall, and F1 scores indicate a challenge in effectively predicting less frequent crime occurrences, a common issue in imbalanced datasets. These metrics highlight areas where our models need refinement.

**Pathways to Improvement:**
Moving forward, we aim to enhance our models through advanced feature engineering, hyperparameter optimization, and potentially incorporating more complex algorithms. Additionally, integrating more contextual data, like socio-economic indicators, could enrich our models' understanding of the underlying patterns in crime data.

